In [135]:
#external libraries
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
import folium
from folium import plugins
from datetime import datetime
%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
#local imports
from func.gdfLoader import gdfLoader

In [137]:
MapDataPath="data/cartePalaiseau.csv"
ModuleMapPath="data/carteModulePalaiseau.csv"

Data_Path="data/manip1.1.csv"

sampling_time=2000#ms #si supérieur à 2000ms il faut implémenter la moyenne dans la liste des rssi dans marker dict list



In [138]:
#load the data
map_gdf=gdfLoader(MapDataPath)
#change the projection to longitude /latitude
map_gdf.to_crs(epsg=4326, inplace=True)
map_gdf.head()

,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ..."


# Add markers of the CartoModule to map

In [139]:
import ast
#load the receiver map for selected map and layer
marker_gdf = gpd.read_file(ModuleMapPath)
#transform the str defining the list in column coordonneesEPSG3857 into a python list
marker_gdf["coordonneesEPSG3857"] = marker_gdf["coordonneesEPSG3857"].apply(ast.literal_eval)
#transform the list of coordinates into points in geometry column
marker_gdf["geometry"] = marker_gdf["coordonneesEPSG3857"].apply(lambda coords: Point(coords))
#drop the column coordonneesEPSG3857 and the layer id
marker_gdf = marker_gdf.drop(columns=["coordonneesEPSG3857","idCouche"])
#set the crs
marker_gdf=marker_gdf.set_crs(crs="epsg:3857", allow_override=True)
# change the projection to lat/lon
marker_gdf=marker_gdf.to_crs(epsg=4326)
marker_gdf.head()




,macModule,geometry
0,A8032A31204E,POINT (2.20079 48.71316)
1,C45BBE39F42A,POINT (2.20110 48.71315)
2,C45BBE39F9AE,POINT (2.20120 48.71314)
3,A8032A311FAA,POINT (2.20072 48.71317)
4,C45BBE39F56A,POINT (2.20117 48.71318)


In [140]:
#to optimize the getting of location of macModule, we create a dictionnary
MacModuleLocation = marker_gdf.set_index("macModule").to_dict()["geometry"]
MacModuleLocation
#it display the approximation of point location in print but it is not the case in the dataframe

{'A8032A31204E': <POINT (2.201 48.713)>,
 'C45BBE39F42A': <POINT (2.201 48.713)>,
 'C45BBE39F9AE': <POINT (2.201 48.713)>,
 'A8032A311FAA': <POINT (2.201 48.713)>,
 'C45BBE39F56A': <POINT (2.201 48.713)>,
 'C45BBE37B346': <POINT (2.201 48.713)>,
 'C45BBE386952': <POINT (2.201 48.713)>,
 'C45BBE386362': <POINT (2.201 48.713)>,
 'C45BBE384CD2': <POINT (2.201 48.713)>,
 'C45BBE39870E': <POINT (2.201 48.713)>,
 'A8032A312052': <POINT (2.201 48.713)>,
 'A8032A31204A': <POINT (2.201 48.713)>,
 'A8032A311F6A': <POINT (2.201 48.713)>,
 'A8032A311F56': <POINT (2.201 48.713)>,
 'C45BBE38FEAA': <POINT (2.201 48.713)>,
 'A8032A30FBFE': <POINT (2.2 48.713)>,
 'A8032A311DAA': <POINT (2.201 48.713)>,
 'A8032A30FB9E': <POINT (2.2 48.713)>,
 'C45BBE39F4D6': <POINT (2.201 48.713)>,
 'A8032A311F96': <POINT (2.201 48.713)>,
 'A8032A311F66': <POINT (2.202 48.713)>,
 'A8032A311FBA': <POINT (2.202 48.713)>,
 'A8032A311F52': <POINT (2.201 48.713)>,
 'A8032A311DBA': <POINT (2.201 48.713)>,
 'A8032A30FBEE': <PO

### Bind the received RSSI Data to the CartoModule Marker on Map

In [141]:
#load the received rssi data
rssi_df = pd.read_csv(Data_Path)
#timestamp is a string, we need to convert it to datetime
rssi_df["timestamp"] = pd.to_datetime(rssi_df["timestamp"])
# set the timestamp as index
rssi_df=rssi_df.sort_values('timestamp')
#drop the duplicates
rssi_df=rssi_df.drop_duplicates()
rssi_df.index = pd.RangeIndex(len(rssi_df.index))


In [142]:
rssi_df['geometry']=rssi_df["macModule"].apply(lambda macModule: MacModuleLocation[macModule])
rssi_df.head()

,timestamp,macModule,rssi,geometry
0,2023-06-01 12:35:46.605,A8032A311DAA,-66,POINT (2.200573795873236 48.713174265469796)
1,2023-06-01 12:35:46.617,A8032A311F6A,-67,POINT (2.200638986440939 48.71317445007082)
2,2023-06-01 12:35:46.693,A8032A311F96,-71,POINT (2.2006869316821076 48.71319455060883)
3,2023-06-01 12:35:48.290,A8032A31204A,-78,POINT (2.2007622901244153 48.71318498422266)
4,2023-06-01 12:35:48.332,A8032A311F56,-70,POINT (2.200612881651723 48.713203074448835)


In [143]:
#create a list of timestamps between min_timestamp and max_timestamp with a step of sampling_time
min_timestamp=rssi_df.iloc[0]['timestamp']+pd.Timedelta(sampling_time, unit="ms")
max_timestamp=rssi_df.iloc[-1]['timestamp']
timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()


In [144]:
def timed_marker_intensity_list(rssi_df, sampling_time):
    min_timestamp=rssi_df.iloc[0]['timestamp']+pd.Timedelta(sampling_time, unit="ms")
    max_timestamp=rssi_df.iloc[-1]['timestamp']
    timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    #implementation with a list of dataframes ouput
    marker_intensity_gdf_list=[]
    sampling_time=pd.Timedelta(sampling_time, unit="ms")
    for current_timestamp in timestamp_list:
        #for each module, we keep only the rssi values between current_timestamp and current_timestamp-sampling_time
        rssi_df_current=rssi_df[(rssi_df["timestamp"] > current_timestamp-sampling_time)&(rssi_df["timestamp"] < current_timestamp)]
        rssi_df_current=rssi_df_current.drop(columns=["timestamp","macModule"]).groupby("macModule").mean()
        marker_intensity_gdf=marker_gdf.merge(rssi_df_current, left_on="macModule", right_on="macModule")
        #mean of the rssi values for each module
        marker_intensity_gdf_list.append(marker_intensity_gdf)
    return(marker_intensity_gdf_list)

In [145]:
def getMarkerDictList(rssi_df, sampling_time, min_timestamp, max_timestamp):
    MarkerDictList = []
    #i in rssi_df.index
    index_count=0
    max_count = len(rssi_df)-1
    timestamp_list=pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    #on parcours la liste des timestamp, on fait en sorte de profiter que les données sont triées par timestamp, pour cela on crée un dict avec les macModule en clé et les rssi en valeur pour chaque intervalle de temps
    for timestamp in timestamp_list:
        markerdict={}
        while index_count!= max_count and rssi_df.iloc[index_count]['timestamp'] < timestamp:
            MacModule=rssi_df.iloc[index_count]['macModule']
            #if key does not exist, we create it and add the rssi value
            if MacModule not in markerdict:
                markerdict[MacModule]=[rssi_df.iloc[index_count]['rssi']]
            #if key exist, we add the rssi value
            else:
        
                markerdict[MacModule].append(rssi_df.iloc[index_count]['rssi'])    
            index_count+=1
        MarkerDictList.append(markerdict)
    return(MarkerDictList)

In [146]:
MarkerDictList=getMarkerDictList(rssi_df, sampling_time, min_timestamp, max_timestamp )

In [147]:

#todo : display the signal intensity on the map must create different feature for each different display change ( green, orange, red, )
def MarkerLayer(MarkerDictList):
    timestamp_list=pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    
    pointList=[]
    MarkerTimestamplist=[]
    rssiList=[]
    for index,timestamp in enumerate(timestamp_list):
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)

        for MarkerDict in MarkerDictList[index]:
                point=MacModuleLocation[MarkerDict]
                pointList.append([point.x,point.y])
                MarkerTimestamplist.append(new_timestamp)
                rssiList.append(str(MarkerDictList[index][MarkerDict]))
    print(pointList)
    print(MarkerTimestamplist)
    print(rssiList)
    
    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPoint',
            'coordinates': pointList
        },
        'properties': {
            'times': MarkerTimestamplist,
            "icon": "circle",
            "popup": rssiList,
            "iconstyle": {
                "fillColor": "red"                        
            }}
        }]
        # pp = folium.Popup("rssi = ",str(MarkerDictList[index][MarkerDict]))
        # feature.add_child(pp)
    print(features)
    MarkerLayer=plugins.TimestampedGeoJson(
                        {"type": "FeatureCollection","features": features},
                        period='PT2S',
                        duration='PT1S',
                       transition_time=sampling_time,
                       auto_play=True)
    return(MarkerLayer)

In [148]:

#issue: display only the one point per timestamp, should redo the way of doing the feature  (list of point and list of timestamp)
def TestMarkerLayer():
    features=[]
    timestamp_list=pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    
    
    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPoint',
            'coordinates': [[2.2007200581389768,48.713166904690254]]
        },
        'properties': {
            'times': ["2023-06-01T12:35:46"],
            "icon": "circle",
            "iconstyle": {
                "fillColor": "red",
                "radius": 100                     
            }}
    }]

    MarkerLayer=plugins.TimestampedGeoJson(
                        {"type": "FeatureCollection","features": features},
                        period='PT2S',
                       transition_time=sampling_time,
                       auto_play=True)
    return(MarkerLayer)

In [149]:
m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap",max_zoom=30, zoom_start=20,crs="EPSG3857")
tooltip = folium.features.GeoJsonTooltip(fields=['ID_element', 'nom'], 
                                        labels=True,
                                        stick=False)
folium.GeoJson(map_gdf, tooltip=tooltip).add_to(m)




In [150]:
marker_layer= MarkerLayer(MarkerDictList)
print(type(marker_layer))
marker_layer.add_to(m)


[[2.200573795873236, 48.713174265469796], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007622901244153, 48.71318498422266], [2.200612881651723, 48.713203074448835], [2.2007200581389768, 48.713166904690254], [2.200573795873236, 48.713174265469796], [2.200612881651723, 48.713203074448835], [2.2007200581389768, 48.713166904690254], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.200612881651723, 48.713203074448835], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007866061683496, 48.713157751163116], [2.2007622901244153, 48.71318498422266], [2.200612881651723, 48.713203074448835], [2.200573795873236, 48.713174265469796], [2.2007866061683496, 48.713157751163116], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007866061683496, 48.713157751163116], [2.2007622901244153, 48.71318498422266], [2.200612881651723, 48.713203074448835], [2.20072005813

In [151]:
folium.CircleMarker([48.713166904690254, 2.2007200581389768], radius=7, color='#FF0000', fill_color='#0080bb').add_to(m)

In [152]:
folium.LayerControl().add_to(m)

In [153]:
m.save('output/marker_map.html')

In [154]:
m

i'm not convinced that heatmap view is interessant

In [155]:
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap,HeatMapWithTime


m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap",max_zoom=30, zoom_start=20,crs="EPSG3857")

tooltip = folium.features.GeoJsonTooltip(fields=['ID_element', 'nom'], 
                                        labels=True,
                                        stick=False)
tooltip2 = folium.features.GeoJsonTooltip(fields=['macModule',], 
                                        labels=True,
                                        stick=False)

folium.GeoJson(map_gdf, tooltip=tooltip).add_to(m)
folium.GeoJson(marker_gdf, tooltip=tooltip2).add_to(m)
#creating the heatmapWithTime
HeatMapWithTime(coord_list_with_time,auto_play=True, radius=50).add_to(m)
folium.LayerControl().add_to(m)

m

NameError: name 'coord_list_with_time' is not defined

In [ ]:

m.save('data/heat_map.html')


In [ ]:
# Function to draw points in the map
def draw_points(map_object, list_of_points, layer_name, line_color, fill_color, text):

    fg = folium.FeatureGroup(name=layer_name)

    for point in list_of_points:
        fg.add_child(
            folium.CircleMarker(
                point,
                radius=1,
                color=line_color,
                fill_color=fill_color,
                popup=(folium.Popup(text)),
            )
        )

    map_object.add_child(fg)

# implementation of low precision localisation

### 1. Naive implementation 

we're looking for 3 points at each sample,
if no 3 receivers received , we take the last 3 received,
if more than 3 receivers received we keep the 3 highest RSSI signal received

In [ ]:
list_of_list_of_points=[]
for i in range(len(timed_marker_intensity_list)):
    list_of_points = []
    marker_intensity_gdf = timed_marker_intensity_list[i]
    if len(marker_intensity_gdf) > 3:
        #keep only the 3 modules with the highest rssi values
        marker_intensity_gdf=marker_intensity_gdf.sort_values(by="rssi", ascending=False).head(3)
    for row in marker_intensity_gdf.iterrows():
        y,x=row[1]["geometry"].y,row[1]["geometry"].x
        longitude,latitude  = transformer.transform(x, y)
        list_of_points.append([latitude,longitude])
    count_backward=1
    while len(list_of_points) < 3 and i-count_backward >= 0:
        points_to_add=3-len(list_of_points)
        marker_intensity_gdf=timed_marker_intensity_list[i-count_backward]
        marker_intensity_gdf=marker_intensity_gdf.sort_values(by="rssi", ascending=False).head(points_to_add)
        for row in marker_intensity_gdf.iterrows():
            y,x=row[1]["geometry"].y,row[1]["geometry"].x
            longitude,latitude  = transformer.transform(x, y)
            if [latitude,longitude] in list_of_points:
                continue
            list_of_points.append([latitude,longitude])
        count_backward+=1    
        
    list_of_list_of_points.append(list_of_points)

for list1 in list_of_list_of_points:
    if len(list1)!=3:
        print("error")
        print(list1)


list_of_list_of_points est une liste définissant les points associés aux 3 récepteurs les plus forts pour chaque temps. on cherche maintenant à afficher les polygones associés dans une map temporelle

In [ ]:

from folium.plugins import TimestampedGeoJson
from datetime import datetime

input_format = '%Y-%m-%d %H:%M:%S.%f'
output_format = '%Y-%m-%dT%H:%M:%S'




def plot_map_and_polygonal_localization(list_of_list_of_points):

    # Create the map used to visualise the locations of the images.

    m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap", zoom_start=22,max_zoom=28,crs="EPSG3857")
    folium.GeoJson(gdf, tooltip=tooltip).add_to(m)
    folium.GeoJson(marker_gdf, tooltip=tooltip2).add_to(m)



    features = []

    for index,triangle in enumerate(list_of_list_of_points):
        timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [triangle]
            },
            'properties': {
                'time': new_timestamp,
                'style': {'color': ''},
                'iconstyle': {
                    'fillColor': '#0000FF',
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 5
                }
            }
        }
        features.append(feature)

    TimestampedGeoJson(features,
                        period='PT1S',
                        duration='PT1S',
                       transition_time=sampling_time,
                       
                       auto_play=True).add_to(m)
    folium.LayerControl().add_to(m)

    m.save('map_with_gps_locations.html')
    return(m)


In [ ]:
print(sampling_time)
plot_map_and_polygonal_localization(list_of_list_of_points)

In [ ]:
print(list_of_list_of_points)

In [ ]:
def polygonlist_to_polygoncenterlist(list_of_list_of_points):
    centerList=[]
    for list1 in list_of_list_of_points:
        n=0
        xlist=0
        ylist=0
        for point in list1:    
            xlist+=point[0]
            ylist+=point[1]
            n+=1
        centerList.append([xlist/n,ylist/n])
    return(centerList)
centerList=polygonlist_to_polygoncenterlist(list_of_list_of_points)
        

In [ ]:
centerList